In [1]:
import os
import nibabel as nib
import numpy as np
import surface_distance
from surface_distance import metrics
import pandas as pd

sCT_dir = r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\sCT_files\segments"
totalseg_dir = r"Z:\FacialDeformation_MPhys\paired_data\nina_abby\UIDQQ0Q00Q910\segments"

sCT_files = {os.path.basename(f): os.path.join(sCT_dir, f)
                for f in os.listdir(sCT_dir) if f.endswith(".nii") or f.endswith(".nii.gz")}
totalseg_files = {os.path.basename(f): os.path.join(totalseg_dir, f)
                  for f in os.listdir(totalseg_dir) if f.endswith(".nii") or f.endswith(".nii.gz")}

common_names = sorted(set(sCT_files.keys()) & set(totalseg_files.keys()))

if not common_names:
    raise ValueError("No matching segmentation filenames found between the two folders.")

results = []

print(f"\nFound {len(common_names)} matching segmentations.\n")

for name in common_names:
    print(f"Processing: {name}")

    img_sCT = nib.load(sCT_files[name])
    img_totalseg = nib.load(totalseg_files[name])

    mask_sCT = img_sCT.get_fdata() > 0
    mask_totalseg = img_totalseg.get_fdata() > 0

    spacing_mm = img_sCT.header.get_zooms()[:3]

    surface_distances = surface_distance.compute_surface_distances(
        mask_sCT, mask_totalseg, spacing_mm
    )

    asd = surface_distance.compute_average_surface_distance(surface_distances)
    hausdorff_100 = surface_distance.compute_robust_hausdorff(surface_distances, 100)
    hausdorff_95 = surface_distance.compute_robust_hausdorff(surface_distances, 95)
    surface_overlap_1mm = surface_distance.compute_surface_overlap_at_tolerance(
        surface_distances, tolerance_mm=1
    )
    surface_dice_1mm = surface_distance.compute_surface_dice_at_tolerance(
        surface_distances, tolerance_mm=1
    )
    volumetric_dice = surface_distance.compute_dice_coefficient(mask_sCT, mask_totalseg)

    results.append({
        "Name": name,
        "ASD_GT→Pred (mm)": float(asd[0]),
        "ASD_Pred→GT (mm)": float(asd[1]),
        "Hausdorff_100 (mm)": float(hausdorff_100),
        "Hausdorff_95 (mm)": float(hausdorff_95),
        "SurfaceOverlap@1mm_GT": float(surface_overlap_1mm[0]),
        "SurfaceOverlap@1mm_Pred": float(surface_overlap_1mm[1]),
        "SurfaceDice@1mm": float(surface_dice_1mm),
        "VolumetricDice": float(volumetric_dice)
    })

df = pd.DataFrame(results)
pd.set_option('display.max_columns', None)
print("\n--- Summary of Metrics ---")
print(df)

# save as csv
output_csv = os.path.join(sCT_dir, "segmentation_comparison_results.csv")
df.to_csv(output_csv, index=False)
print(f"\nSaved results to: {output_csv}")



Found 7 matching segmentations.

Processing: head.nii.gz


ValueError: operands could not be broadcast together with shapes (336,336,240) (312,312,150) 